In [5]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium_stealth import stealth

In [18]:
from pathlib import Path
import os

WindowsPath('c:/Users/V61XNRQ/Desktop/PERS/bundestag')

In [20]:
CHROMEDRIVER_PATH = Path("executables/chromedriver.exe")
DOWNLOAD_FOLDER_NAME = "downloads"
BASE_URL = "https://www.bundestag.de/services/opendata"

WindowsPath('c:/Users/V61XNRQ/Desktop/PERS/bundestag/downloads')

In [9]:
if not Path(DOWNLOAD_FOLDER_NAME).exists():
    Path(DOWNLOAD_FOLDER_NAME).mkdir()

In [22]:
class Webdriver_Handler:
    def __init__(self):
        options = webdriver.ChromeOptions()
        options.add_argument('disable-infobars')
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_experimental_option('excludeSwitches', ['enable-logging'])
        prefs = {
            "profile.default_content_settings.popups": 0,    
            "download.default_directory": str(Path(os.getcwd(), DOWNLOAD_FOLDER_NAME)), ### Set the path accordingly
            "download.prompt_for_download": False, ## change the downpath accordingly
            "download.directory_upgrade": True
        }
        options.add_experimental_option("prefs", prefs)

        self.timeout = 3       

        self.wd = webdriver.Chrome(service=Service(CHROMEDRIVER_PATH), options=options)
        
        self.wd.implicitly_wait(self.timeout)

        stealth(
            self.wd,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True,
        )
    
    def get_plenarprotokolle(self):
        self.wd.get(BASE_URL)

        download_divs = self.wd.find_elements(By.XPATH, "//div[@class='container-fluid bt-module-content-wrap']")

        download_divs_titles = [div.find_element(By.XPATH, ".//h2").text for div in download_divs]
        print(f"Possible download divs: {[h2 for h2 in download_divs_titles]}")
        slide_arrows = [div.find_element(By.XPATH, ".//button[@class='slick-next slick-arrow']") for div in download_divs]
        print("slideable:", [slide_arrow.get_attribute("aria-disabled") == "false" for slide_arrow in slide_arrows])
        xml_links = [div.find_elements(By.XPATH, """.//a[contains(@href,"xml") and contains(@title,"XML")]""") for div in download_divs]
        print("number_xml", [len(xml_link) for xml_link in xml_links])

        for download_div in download_divs:
            div_header = download_div.find_element(By.XPATH, ".//h2").text
            print("Downloading", div_header)
            all_download_links = []
            
            link_elements = download_div.find_elements(By.XPATH, """.//a[contains(@href,"xml") and contains(@title,"XML")]""")
            xml_links = [link.get_attribute("href") for link in link_elements]
            all_download_links += xml_links

            if download_div.find_element(By.XPATH, ".//button[@class='slick-next slick-arrow']") is not None:
                print("Element is slideable")
                while download_div.find_element(By.XPATH, ".//button[@class='slick-next slick-arrow']").get_attribute("aria-disabled") == "false":
                    print("Next slide")
                    download_div.find_element(By.XPATH, ".//button[@class='slick-next slick-arrow']").click()
                    time.sleep(1)
                    link_elements = download_div.find_elements(By.XPATH, """.//a[contains(@href,"xml") and contains(@title,"XML")]""")
                    xml_links = [link.get_attribute("href") for link in link_elements]
                    all_download_links += xml_links
        
            print(f"All download links for {div_header}: {all_download_links}")
